In [1]:
from langchain.agents import initialize_agent, Tool, AgentType
from langchain.tools import StructuredTool
from langchain.chat_models import ChatOpenAI
import os
import json
from apitable import Apitable
import tiktoken
from typing import Optional
from pydantic import BaseModel, Field
from typing import Dict, List

In [2]:
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613")
apitable_api_token = os.getenv("APITABLE_API_TOKEN")
apitable = Apitable(token=apitable_api_token)

In [3]:
class SortCondition(BaseModel):
    field: str = Field(description="Field name")
    order: str = Field(description="Sort order", enum=["desc", "asc"])


class RecordQueryCondition(BaseModel):
    datasheet_id: str = Field(description="The ID of the datasheet to retrieve records from.")
    filter_condition: Optional[Dict[str, str]] = Field(
        description="""
            Find records that meet specific conditions.
            This object should contain a key-value pair where the key is the field name and the value is the lookup value. For instance: {"title": "test"}.
            """
    )
    sort_condition: Optional[List[SortCondition]] = Field(min_items=1,
        description="Sort returned records by specific field"
    )
    maxRecords_condition: Optional[int] = Field(
        description="Limit the number of returned values."
    )


def trans_key(field_key_map, key: str):
    """
    When there is a field mapping, convert the mapped key to the actual key
    """
    if key in ["_id", "recordId"]:
        return key
    if field_key_map:
        _key = field_key_map.get(key, key)
        return _key
    return key

def query_parse(field_key_map, **kwargs) -> str:
    query_list = []
    for k, v in kwargs.items():
        # Handling null
        if v is None:
            v = "BLANK()"
        # Handling string
        elif isinstance(v, str):
            v = f'"{v}"'
        elif isinstance(v, bool):
            v = "TRUE()" if v else "FALSE()"
        # Handling array type values, multiple select, members?
        elif isinstance(v, list):
            v = f'"{", ".join(v)}"'
        query_list.append(f"{{{trans_key(field_key_map, k)}}}={v}")
    if len(query_list) == 1:
        return query_list[0]
    else:
        qs = ",".join(query_list)
        return f"AND({qs})"

def count_tokens(s):
    return len(enc.encode(s))

enc = tiktoken.encoding_for_model("text-davinci-003")

def get_spaces(question: str):
    spaces = apitable.spaces.all()
    return [json.loads(space.json()) for space in spaces]

def get_nodes(space_id: str):
    nodes = apitable.space(space_id=space_id).nodes.all()
    return [json.loads(node.json()) for node in nodes]

def get_fields(datasheet_id: str):
    fields = apitable.datasheet(datasheet_id).fields.all()
    return [json.loads(field.json()) for field in fields]

def create_fields(space_id: str, datasheet_id: str, field_data: dict[str, str]):
    field = (
        apitable.space(space_id)
        .datasheet(datasheet_id)
        .fields.create(field_data)
    )
    return field.json()

def get_records(datasheet_id:str, filter_condition: Optional[dict]= None, sort_condition: Optional[list] = None, maxRecords_condition: Optional[int] = None):
    dst = apitable.datasheet(datasheet_id)
    query_kwargs = {}
    if filter_condition:
        query_formula = query_parse(filter_condition)
        query_kwargs["filterByFormula"] = query_formula
    if sort_condition:
        query_kwargs["sort"] = sort_condition
    if maxRecords_condition:
        query_kwargs["maxRecords"] = maxRecords_condition
    records = dst.records.all(**query_kwargs)
    parsed_records = [record.json() for record in records]
    # return parsed_records
    if count_tokens(str(parsed_records)) > 1000:
        parsed_records_str = (
            "Found "
            + str(len(parsed_records))
            + " records, too many to show. Try to limit"
        )
        return parsed_records_str
    else:
        return parsed_records
    
tools = [
    Tool(
        name = "get_spaces",
        func=get_spaces,
        description="useful for accessing all spaces the user has access to. Input should be in the form of a question containing full context",
    ),
    Tool(
        name="get_nodes",
        func=get_nodes,
        description="useful for retrieving all file nodes in a space"
    ),
    Tool(
        name="get_fields",
        func=get_fields,
        description="useful for retrieving fields in a datasheet, if you need get"
    ),
    # Tool(
    #     name="create_fields",
    #     func=create_fields.run,
    #     description="useful for when you need to answer questions about FooBar. Input should be in the form of a question containing full context"
    # ),
    # Tool(
    #     name="get_records",
    #     func=get_records,
    #     description="""
    #     useful for retrieving data in a datasheet
    #     filter_condition should contain a key-value pair where the key is the field name and the value is the lookup value.
    #     sort_condition should contain a list of object where each dictionary contains a key-value pair where the key is the field name and the value is asc or desc.
    #     """,
    #     args_schema=RecordQueryCondition
    # )
    StructuredTool(func=get_records, name="get_records", description="useful for retrieving data in a datasheet", args_schema=RecordQueryCondition)
]

mrkl = initialize_agent(tools, llm, agent=AgentType.OPENAI_FUNCTIONS, verbose=True)

In [4]:
mrkl.run("What spaces do I have?")



> Entering new  chain...

Invoking: `get_spaces` with `What spaces do I have?`


[{'id': 'spctqtTZpssYw', 'name': "xukecheng's Space", 'isAdmin': True}, {'id': 'spcS0eZxZ8mSA', 'name': 'APITable Ltd.', 'isAdmin': None}, {'id': 'spcGSVizcwRYF', 'name': 'Gmail Space', 'isAdmin': None}, {'id': 'spcV5VkzU71Lf', 'name': 'test03-bug验证升级', 'isAdmin': None}]You have the following spaces:

1. xukecheng's Space
2. APITable Ltd.
3. Gmail Space
4. test03-bug验证升级

Let me know if you need any further assistance.

> Finished chain.


"You have the following spaces:\n\n1. xukecheng's Space\n2. APITable Ltd.\n3. Gmail Space\n4. test03-bug验证升级\n\nLet me know if you need any further assistance."

In [5]:
mrkl.run("Tell me the latest value of APITable MAU in xukecheng's space")



> Entering new  chain...

Invoking: `get_spaces` with `xukecheng`


[{'id': 'spctqtTZpssYw', 'name': "xukecheng's Space", 'isAdmin': True}, {'id': 'spcS0eZxZ8mSA', 'name': 'APITable Ltd.', 'isAdmin': None}, {'id': 'spcGSVizcwRYF', 'name': 'Gmail Space', 'isAdmin': None}, {'id': 'spcV5VkzU71Lf', 'name': 'test03-bug验证升级', 'isAdmin': None}]
Invoking: `get_nodes` with `spctqtTZpssYw`


[{'id': 'dstZTEueDaWedFWgbh', 'name': 'Project management', 'type': 'Datasheet', 'icon': '', 'isFav': False}, {'id': 'fodHl5i2LeMBH', 'name': 'Make Test', 'type': 'Folder', 'icon': '', 'isFav': False}, {'id': 'fomh7eepvbqkBrwCvW', 'name': 'Form', 'type': 'Form', 'icon': '', 'isFav': False}, {'id': 'dstFQKumRsAp4p5RBE', 'name': 'AITEST', 'type': 'Datasheet', 'icon': '', 'isFav': False}, {'id': 'dstWuVdBJNDzpjn8ck', 'name': 'email_oss', 'type': 'Datasheet', 'icon': '', 'isFav': False}, {'id': 'dsti6VpNpuKQpHVSnh', 'name': 'APITable MAUs', 'type': 'Datasheet', 'icon': '', 'isFav': False}]
Invoking: `get_recor

ValidationError: 1 validation error for RecordQueryCondition
sort_condition -> 0 -> order
  field required (type=value_error.missing)